# Mosaic

Exploration of mosaicking.

# Imports

In [ ]:
# Native python
import os

In [ ]:
# External
import numpy as np
import pandas as pd
import sklearn.model_selection
import pyproj

In [ ]:
# Plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [ ]:
# Custom scripts
from nitelite_mapmaker import mapmaker, georeference

# Settings

In [ ]:
settings = dict(
    # Data architecture
    flight_name = '220513-FH135',
    data_dir = '/Users/Shared/data/nitelite',
    google_drive_dir = '/Users/zhafensaavedra/Google Drive/Shared drives/NITELite/Data & Analysis',
    flight_subdir = 'Old NITELite Flights/220513-FH135',
    reffed_subdir = 'QGIS FH135/FH135 Main Project/Main Geo Files',
    img_log_filename = 'image.log',
    imu_log_filename = 'OBC/PresIMULog.csv',
    gps_log_filename = 'OBC/GPSLog.csv',
      
    # Choices for what to process
    camera_num = 1,
    test_size = 0.2,
    
    # Data filter choices
    gyro_mag_cut = 0.5, # Corresponds to ~84th percentile
    # percent_for_landed = 95.,
    percent_for_cruising = 85.,
    # mult_of_std_for_steady = 2.,
    # rolling_window_in_min = 1.,
    
    # Mosaicking choices
    allotted_memory = 2., # In GB
    cart_crs_code = 'EPSG:3857', # Google maps
    latlon_crs_code = 'EPSG:4326', # WGS84
    n_tiles_guess = 16,
)

# Set Up

## Settings Parsing

In [ ]:
# The camera has an according long number
settings['camera_long_num'] = settings['camera_num'] + 23085686

In [ ]:
# Data architecture processing
settings['image_dir'] = os.path.join(
    settings['data_dir'],
    'images',
    settings['flight_name'],
    str(settings['camera_long_num'])
)
settings['metadata_dir'] = os.path.join(
    settings['google_drive_dir'],
    settings['flight_subdir'],
    'data',
)
settings['reffed_dir'] = os.path.join(
    settings['google_drive_dir'],
    settings['reffed_subdir'],
)

## Object Creation and Preprocessing

In [ ]:
# Create the main mapmaker object
mm = mapmaker.Mapmaker(
    image_dir=settings['image_dir'],
    img_log_fp=os.path.join(settings['metadata_dir'], settings['img_log_filename']),
    imu_log_fp=os.path.join(settings['metadata_dir'], settings['imu_log_filename']),
    gps_log_fp=os.path.join(settings['metadata_dir'], settings['gps_log_filename']),
)

In [ ]:
# General metadata loading
mm.prep()

In [ ]:
# Manually-georeferenced metadata
_ = mm.flight.get_manually_georeferenced_filepaths(
    settings['reffed_dir']
)

In [ ]:
# Establish CRS and conversions
cart_crs = pyproj.CRS(settings['cart_crs_code'])
latlon_crs = pyproj.CRS(settings['latlon_crs_code'])
cart_to_latlon = pyproj.Transformer.from_crs(cart_crs, latlon_crs)
latlon_to_cart = pyproj.Transformer.from_crs(latlon_crs, cart_crs)

# Exploration

In [ ]:
metadata = mm.flight.metadata

## Determine How Many Images We Can Load

In [ ]:
# Image locations
metadata['filepath'] = metadata['filename'].apply(lambda x: os.path.join(settings['image_dir'], os.path.basename(x)))

In [ ]:
def get_size_in_GB(fp):
    try:
        return os.path.getsize(fp) / 1024**3
    except FileNotFoundError:
        return np.nan

In [ ]:
filesizes = metadata['filepath'].apply(get_size_in_GB)
median_filesize = np.nanmedian(filesizes)
n_files_in_memory = int(settings['allotted_memory'] // median_filesize)

## Determine What Images Are Valid

In [ ]:
metadata['valid'] = True

### Cruise Altitude

In [ ]:
h_max = metadata['mAltitude'].max()
h_min = metadata['mAltitude'].min()
h_diff = h_max - h_min

h_cruising = h_min + settings['percent_for_cruising'] / 100. * h_diff

In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.scatter(
    metadata['timestamp'],
    metadata['mAltitude'],
)

ax.axhline(h_cruising)

In [ ]:
metadata.loc[metadata['mAltitude']<h_cruising, 'valid'] = False

### Movement

In [ ]:
# Magnitude of Gyro
metadata['imuGyroMag'] = np.sqrt((metadata[['imuGyroX','imuGyroY','imuGyroZ']]**2.).sum(axis='columns'))

In [ ]:
# Fancy method for movement


# # Select cruise data
# cruise_data = metadata.loc[metadata['flight_phase'] == 'cruise']
# cruise_data = cruise_data.set_index('timestamp')

# # Get rolling deviation
# cruise_rolling = cruise_data.rolling(window=pd.Timedelta(settings['rolling_window_in_min'], 'min'))
# cruise_rolling_std = cruise_rolling.std(numeric_only=True)

# # Identify and store steady data
# cruise_data.loc[:,'is_steady'] = cruise_rolling_std['imuGyroMag'] < settings['mult_of_std_for_steady'] * np.nanmedian(cruise_rolling_std['imuGyroMag'])
# cruise_rolling_std.loc[:,'is_steady'] = cruise_data['is_steady']
# metadata['is_steady'] = False
# metadata.loc[metadata['flight_phase'] == 'cruise','is_steady'] = cruise_data['is_steady'].values



In [ ]:
# Must not be moving too fast
metadata.loc[metadata['imuGyroMag'] > settings['gyro_mag_cut'], 'valid'] = False

### Camera Number

In [ ]:
metadata.loc[metadata['camera_num'] != settings['camera_num'], 'valid'] = False

### Select Data

In [ ]:
selected = metadata.loc[metadata['valid']].copy()

## Make Tiles

### Determine Tilesize

In [ ]:
# Convert to get sensor coords
selected['sensor_x'], selected['sensor_y'] = latlon_to_cart.transform(selected['GPSLat'], selected['GPSLong'])

In [ ]:
x_bins = int(np.sqrt(settings['n_tiles_guess']))
y_bins = x_bins

In [ ]:
# Refine until we can hold all in memory
while True:

    # Initial guess for tiling
    hist2d, x_edges, y_edges = np.histogram2d(
        selected['sensor_x'],
        selected['sensor_y'],
        (x_bins, y_bins),
    )
    hist_max = hist2d.max()
    
    if hist_max < n_files_in_memory:
        break
        
    # Determine tile size based on max density and number of files allowed in memory
    max_surface_density = hist_max / (x_edges[1] - x_edges[0]) / (y_edges[1] - y_edges[0])
    tile_area = n_files_in_memory / max_surface_density
    tile_length = np.sqrt(tile_area)
    x_bins = np.arange(selected['sensor_x'].min(), selected['sensor_x'].max() + tile_length, tile_length)
    y_bins = np.arange(selected['sensor_y'].min(), selected['sensor_y'].max() + tile_length, tile_length)

In [ ]:
fig = plt.figure(figsize=(16,8))
ax_dict = fig.subplot_mosaic([['scatter', 'hist_guess', 'hist']])

ax = ax_dict['scatter']
ax.scatter(
    selected['sensor_x'],
    selected['sensor_y'],
)

ax = ax_dict['hist_guess']
hist2d_guess, x_edges, y_edges, img_view = ax.hist2d(
    selected['sensor_x'],
    selected['sensor_y'],
    (int(np.sqrt(settings['n_tiles_guess'])), int(np.sqrt(settings['n_tiles_guess']))),
)
plt.colorbar(img_view, ax=ax)

ax = ax_dict['hist']
hist2d, x_edges, y_edges, img_view = ax.hist2d(
    selected['sensor_x'],
    selected['sensor_y'],
    (x_bins, y_bins),
)
plt.colorbar(img_view, ax=ax)

for ax_key, ax in ax_dict.items():
    ax.set_aspect('equal')